# Preparation

* label images in vott
* make dataset in roboflow
* upload labelled images to dataset
* export dataset to yolov5

# Method 1: google collab training
When working in google colab: change runtime type in google collab to GPU in hardware accelerator (arrow next to connect in the right corner -> view resources -> change runtime type)

Note: you cannot run google collab indefinitely, normally max 12 hours and you cannot be idle more than 3 hours, also don't close your browser else everything is gone

In [1]:
# check which GPU u got, most of the time it's a K80
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2270111566349575632
 xla_global_id: -1]

### Scroll down to train your model
When training is done you can export your model's weights below

### Export model
Export your model's weights for future use

Note: you may have to change the path to latest run

In [ ]:
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')

# Method 2: local training
When working local (windows 10) and have a nvidia GPU: make a venv + install nvidia cuda + install pytorch for cuda

https://pytorch.org/get-started/locally/

In [ ]:
# python -m venv venv
# .\venv\Scripts\activate
# pip install ipykernel
# python -m ipykernel install --user --name venv --display-name "Python my venv"
# (optional) reload window
# select venv as kernel

In [ ]:
# this can take a while and has no output in jupter notebook during install. If you want output install in terminal
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [6]:
# check if pytorch detects your GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_capability(0))
print(torch.cuda.get_device_properties(0))

True
1
0
NVIDIA GeForce GTX 1660 Ti
(7, 5)
_CudaDeviceProperties(name='NVIDIA GeForce GTX 1660 Ti', major=7, minor=5, total_memory=6143MB, multi_processor_count=24)


# Training
Next sections is for google collab AND local training

### Pull yolov5 enviroment

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5
%pip install -r yolov5\requirements.txt
%pip install roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

### (optional) install wandb to view perfomance while model is training

In [5]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()
# don't forget to authorize login

You should consider upgrading via the 'c:\Users\felix\Documents\Howest\2S2\TESTING\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: platteaufelix. Use `wandb login --relogin` to force relogin


True

### Pull dataset images
Generate new version in roboflow and export to yolov5 pytorch and paste download code

In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="lBeRX0ddTHzL4Dk8rJ1l")
project = rf.workspace("dlc-ai").project("labelled-frames")
dataset = project.version(2).download("yolov5")

ModuleNotFoundError: No module named 'roboflow'

### Train model
* choose a model https://pytorch.org/hub/ultralytics_yolov5/
* most of the time yolov5s or yolov5m is good enough
* start with 300 epoch if you can
* set image size at resolution of photo when it needs to detect small objects
* set batch at -1 to use max allowed
* (optional) if you've installed wandb: check progress at https://wandb.ai/home

In [ ]:
!python train.py --img 1080 --batch -1 --epochs 300 --data labelled-frames-1/data.yaml --weights yolov5s.pt

In [ ]:
# when training is done: plot results
from utils.plots import plot_results 
plot_results('path/to/results.csv')

### Test
(local) test model on test images in dataset

(google collab) Can be tested in the cloud but also local if you export model from google collab, see method 1: export

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('./runs/labelled-frames-1/test/images*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

### Detect
can take a while

(local) change link to source of file you want to detect and run

(google collab) upload video (, change path) and run

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 1080 --source video.mp4